In [3]:
from Funcs.Utility import *

In [4]:
p = os.path.join(PATH_INTERMEDIATE, 'feat',f'stress-fixed.pkl')

X, y, groups, t, datetimes = load(p)
##############################################
# #Remove users with extreme label distribution
# # Create a DataFrame from y, groups, t, datetimes
# info_df = pd.DataFrame({
#     'y': y,
#     'groups': groups,
#     't': t,
#     'datetimes': pd.to_datetime(datetimes)  # assuming 'datetimes' needs conversion to datetime
# })

# # Calculate majority/minority ratio for each group
# def calculate_ratio(group):
#     counts = group['y'].value_counts()
#     if len(counts) > 1:
#         majority = counts.max()
#         minority = counts.min()
#         ratio = majority / minority
#     else:
#         ratio = np.inf  # Infinite ratio if there's no minority class
#     return ratio

# # Apply the function per group
# group_ratios = info_df.groupby('groups').apply(calculate_ratio)

# # Filter groups based on the ratio
# filtered_groups = group_ratios[group_ratios <= 4].index

# # Filter the original DataFrame 'info_df' to remove skewed groups
# filtered_info = info_df[info_df['groups'].isin(filtered_groups)]

# # Use the indices of the filtered info to refine 'X'
# X_filtered = X.loc[filtered_info.index]

# # Extracting other arrays from the filtered info
# y_filtered = filtered_info['y'].values
# groups_filtered = filtered_info['groups'].values
# t_filtered = filtered_info['t'].values
# datetimes_filtered = filtered_info['datetimes'].values

# X, y, groups, t, datetimes = X_filtered, y_filtered, groups_filtered, t_filtered, datetimes_filtered

# Now 'X_filtered', 'y_filtered', 'groups_filtered', 't_filtered', 'datetimes_filtered'
# are ready to be used for further analysis or modeling
################################################
# #Remove neutral state samples
# y =  LABELS_PROC['stressLevel'].to_numpy()

# # Create a mask that selects all samples where y is not equal to 3 (neutral state)
# mask = y != 3

# # Apply this mask to filter out the neutral samples from all arrays
# X_filtered = X[mask]  # X is a DataFrame, it uses boolean indexing directly
# y_filtered = y[mask]  # y, groups, t, datetimes are numpy arrays or similar structures
# groups_filtered = groups[mask]
# t_filtered = t[mask]
# datetimes_filtered = datetimes[mask]

# y = (y_filtered > 3).astype(int)
# X = X_filtered
# groups = groups_filtered
# t = t_filtered
# datetimes = datetimes_filtered

################################################
#Use mean threshold for all users (only training set,\ 
#we need to use raw value and binarize after data splitting)
# y =  LABELS_PROC['stressLevel'].to_numpy()
#Use user speicifc mean threshold
# y =LABELS_PROC['stress_user_mean'].to_numpy()
#Use fixed threshold
#         y =LABELS_PROC['stress_fixed'].to_numpy()
#Use three categories (fixed threshold) 
#        y =LABELS_PROC['stress_fixed_tri'].to_numpy()


#The following code is designed for reordering for the sake of time series split 
#################################################
# Create a DataFrame with user_id and datetime

df = pd.DataFrame({'user_id': groups, 'datetime': datetimes, 'label': y})

# df_merged = pd.merge(df, X, left_index=True, right_index=True)
df_merged = pd.merge(df, X, left_index=True, right_index=True)

# Normalize the datetime for each user only needed for timeseries split/groupk partil personalization
#         df_merged['datetime'] = df_merged.groupby('user_id')['datetime'].transform(lambda x: x - x.min())
# df_merged['datetime'] = df_merged.groupby('user_id')['datetime'].transform(lambda x: x - x.min().normalize())

# Sort the DataFrame by datetime
df_merged = df_merged.sort_values(by=['user_id', 'datetime'])
# df_merged = df_merged.sort_values(by=['datetime'])

# Shuffle the DataFrame
# df_merged = df_merged.sample(frac=1, random_state=RANDOM_STATE)

# Update groups and datetimes
groups = df_merged['user_id'].to_numpy()
datetimes = df_merged['datetime'].to_numpy()  
y = df_merged['label'].to_numpy()
X = df_merged.drop(columns=['user_id', 'datetime', 'label'])

#The following code is for shuffling the temporal order for all users
########################################################

# # Assuming 'groups', 'datetimes', 'y', and 'X' are already defined and loaded
# # Create a DataFrame with user_id, datetime, and label
# df = pd.DataFrame({
#     'user_id': groups,
#     'datetime': datetimes,
#     'label': y
# })

# # Merge the new DataFrame with the features DataFrame 'X'
# # Ensure 'X' is indexed the same way as 'groups', 'datetimes', and 'y'
# df_merged = pd.merge(df, X, left_index=True, right_index=True)

# # Shuffle the DataFrame
# # This disregards the temporal ordering completely and randomizes all entries
# df_merged = df_merged.sample(frac=1, random_state=42)  # Use a fixed seed for reproducibility

# # Extract the shuffled 'groups', 'datetimes', 'y', and 'X' from the shuffled DataFrame
# groups_shuffled = df_merged['user_id'].to_numpy()
# datetimes_shuffled = df_merged['datetime'].to_numpy()
# y_shuffled = df_merged['label'].to_numpy()
# X_shuffled = df_merged.drop(columns=['user_id', 'datetime', 'label'])

# # Optionally, you can convert 'X_shuffled' back to the correct type if it needs to be a DataFrame
# X_shuffled = pd.DataFrame(X_shuffled, columns=X.columns)

# X, y, groups, datetimes = X_shuffled, y_shuffled, groups_shuffled, datetimes_shuffled


#The following code is for only using 1st day
###########################################
# filtered_df = pd.read_csv(os.path.join(PATH_INTERMEDIATE,'exclude_1st_day.csv'),index_col=0)
# # filtered_df = pd.read_csv(os.path.join(PATH_INTERMEDIATE,'exclude_1st_week.csv'),index_col=0)
# X_filtered = X[~X.index.isin(filtered_df.index)]
# y_series = pd.Series(y, index=X.index)
# y_filtered = y_series[~y_series.index.isin(filtered_df.index)]
# y_filtered = y_filtered.values
# groups_series = pd.Series(groups, index=X.index)
# groups_filtered = groups_series[~groups_series.index.isin(filtered_df.index)]
# groups_filtered = groups_filtered.values
# X,y, groups=X_filtered,y_filtered, groups_filtered
# #The following code is for excluding using 1st day
# ###########################################
# # filtered_df = pd.read_csv(os.path.join(PATH_INTERMEDIATE,'exclude_1st_week.csv'),index_col=0)
# filtered_df = pd.read_csv(os.path.join(PATH_INTERMEDIATE,'exclude_1st_day.csv'),index_col=0)
# X_filtered = X[X.index.isin(filtered_df.index)]
# y_series = pd.Series(y, index=X.index)
# y_filtered = y_series[y_series.index.isin(filtered_df.index)]
# y_filtered = y_filtered.values
# groups_series = pd.Series(groups, index=X.index)
# groups_filtered = groups_series[groups_series.index.isin(filtered_df.index)]
# groups_filtered = groups_filtered.values
# datetimes_series = pd.Series(datetimes, index=X.index)
# datetimes_filtered = datetimes_series[datetimes_series.index.isin(filtered_df.index)]
# datetimes_filtered = datetimes_filtered.values
# X,y, groups, datetimes=X_filtered,y_filtered, groups_filtered, datetimes_filtered


###########################################
#The following code is for similar-user model
###########################################
#         similar_user = pd.read_csv(os.path.join(PATH_INTERMEDIATE,  'similar_user.csv'))
#         cluster_label = similar_user['cluster'].value_counts().index[0] #N number clusters
#         similar_users_in_cluster = similar_user[similar_user['cluster'] == cluster_label]['pcode']

#         # Check if each value in 'groups' is in 'similar_users_in_cluster'
#         mask = np.isin(groups, similar_users_in_cluster)

#         # Filter 'groups' based on the mask
#         filtered_groups = groups[mask]
#         # Filter 'X' and 'y' based on the mask
#         X_filtered = X[mask]
#         y_filtered = y[mask]
#         X,y, groups=X_filtered,y_filtered, filtered_groups
###########################################
#Remove low frequency features
#         mask = ['CAE#', 'MED#', 'ONF#', 'PWS#', 'RNG#','MSG#' ]
#         X = X.loc[:, [all(m not in str(x) for m in mask) for x in X.columns]]

#Divide the features into different categories
feat_current = X.loc[:,[('#VAL' in str(x)) or ('ESM#LastLabel' in str(x)) for x in X.keys()]]  
feat_dsc = X.loc[:,[('#DSC' in str(x))  for x in X.keys()]]  
feat_yesterday = X.loc[:,[('Yesterday' in str(x))  for x in X.keys()]]  
feat_today = X.loc[:,[('Today' in str(x))  for x in X.keys()]]  
feat_sleep = X.loc[:,[('Sleep' in str(x))  for x in X.keys()]]  
feat_time = X.loc[:,[('Time' in str(x))  for x in X.keys()]]  
feat_pif = X.loc[:,[('PIF' in str(x))  for x in X.keys()]]  
feat_ImmediatePast = X.loc[:,[('ImmediatePast_15' in str(x))  for x in X.keys()]]
#Divide the time window features into sensor/past stress label
feat_current_sensor = X.loc[:,[('#VAL' in str(x))  for x in X.keys()]]  
feat_current_ESM = X.loc[:,[('ESM#LastLabel' in str(x)) for x in X.keys()]]  
feat_ImmediatePast_sensor = feat_ImmediatePast.loc[:,[('ESM' not in str(x)) for x in feat_ImmediatePast.keys()]]  
feat_ImmediatePast_ESM = feat_ImmediatePast.loc[:,[('ESM'  in str(x)) for x in feat_ImmediatePast.keys()]]  
feat_today_sensor = feat_today.loc[:,[('ESM' not in str(x))  for x in feat_today.keys()]]  
feat_today_ESM = feat_today.loc[:,[('ESM'  in str(x)) for x in feat_today.keys()]]  
feat_yesterday_sensor = feat_yesterday.loc[:,[('ESM' not in str(x)) for x in feat_yesterday.keys()]]  
feat_yesterday_ESM = feat_yesterday.loc[:,[('ESM'  in str(x)) for x in feat_yesterday.keys()]]



#Prepare the final feature set
feat_baseline = pd.concat([ feat_time,feat_dsc,feat_current_sensor, feat_ImmediatePast_sensor],axis=1)
#The following code is for calculating aggregated features
########################################################################
# # Define a function to split the column name into sensor and attribute
# def split_column_name(col_name):
#     parts = col_name.rsplit("#", 1)  # Split on last occurrence of '#'
#     return parts[0]  # This gives you 'Sensor#Attribute'

# # Get a list of unique sensor-attribute combinations
# df=feat_today_sensor
# sensor_attributes = df.columns.map(split_column_name).unique()

# # Create a list to hold the aggregated results
# agg_results = []

# # Loop over each sensor-attribute, select the appropriate columns, compute the mean and std
# for sensor_attribute in sensor_attributes:
#     # Select columns for this sensor-attribute
#     cols_to_aggregate = [col for col in df.columns if col.startswith(sensor_attribute)]
#     # Compute the mean and std and store in the new DataFrame
#     agg_results.append(df[cols_to_aggregate].mean(axis=1).rename(sensor_attribute + '|'+ 'MEAN'))
#     agg_results.append(df[cols_to_aggregate].std(axis=1).rename(sensor_attribute + '|'+'STD'))

# # Concatenate all the results into a single DataFrame
# agg_feature = pd.concat(agg_results, axis=1)

######################################################################
feat_final = pd.concat([feat_baseline],axis=1)

#         # Fill NaN values with zeros
#         feat_final = feat_final.fillna(0)

#         # Find the maximum non-infinity value and minimum non-negative infinity value across the entire dataframe
#         max_val = feat_final[feat_final != np.inf].max().max()
#         min_val = feat_final[feat_final != -np.inf].min().min()

#         # Replace positive and negative infinity values
#         feat_final.replace(np.inf, max_val, inplace=True)
#         feat_final.replace(-np.inf, min_val, inplace=True)

X = feat_final
cats = X.columns[X.dtypes == bool]

In [9]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator
from sklearn.model_selection import StratifiedShuffleSplit
from typing import Union


class EvXGBClassifier(BaseEstimator):
    def __init__(
        self,
        eval_size=None,
        eval_metric='logloss',
        early_stopping_rounds=10,
        random_state=None,
        **kwargs
        ):
        self.random_state = random_state
        self.eval_size = eval_size
        self.eval_metric = eval_metric
        self.early_stopping_rounds = early_stopping_rounds
        self.model = XGBClassifier(
            random_state=self.random_state,
            eval_metric=self.eval_metric,
            early_stopping_rounds=self.early_stopping_rounds,
            **kwargs
        )

    @property
    def classes_(self):
        return self.model.classes_

    @property
    def feature_importances_(self):
        return self.model.feature_importances_
    
    @property
    def feature_names_in_(self):
        return self.model.feature_names_in_

    def fit(self, X: Union[pd.DataFrame, np.ndarray], y: np.ndarray):
        if self.eval_size:
            splitter = StratifiedShuffleSplit(random_state=self.random_state, test_size=self.eval_size)
            I_train, I_eval = next(splitter.split(X, y))
            if isinstance(X, pd.DataFrame):
                X_train, y_train = X.iloc[I_train, :], y[I_train]
                X_eval, y_eval = X.iloc[I_eval, :], y[I_eval]
            else:
                X_train, y_train = X[I_train, :], y[I_train]
                X_eval, y_eval = X[I_eval, :], y[I_eval]
                
            self.model = self.model.fit(
                X=X_train, y=y_train, 
                eval_set=[(X_eval, y_eval)],
                verbose=False
            )
        else:
            self.model = self.model.fit(X=X, y=y, verbose=False)
        # After fitting, store the best iteration
        # self.best_iteration_ = self.model.get_booster().best_iteration
        return self

    def predict(self, X: pd.DataFrame):
        return self.model.predict(X)
        # return self.model.predict(X, iteration_range=(0, self.best_iteration_ + 1))

    def predict_proba(self, X: pd.DataFrame):
        return self.model.predict_proba(X)
        # return self.model.predict_proba(X, iteration_range=(0, self.best_iteration_ + 1))

In [5]:
import os
import pandas as pd
import numpy as np
import time
import traceback
from sklearn.linear_model import LogisticRegression
import ray
from sklearn.base import clone
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score
from dataclasses import dataclass

@dataclass
class FoldResult:
    name: str
    metrics: dict
    duration: float

def log(message: str):
    print(message)  # Simple logging to stdout or enhance as needed

@ray.remote
def train_fold(dir_result: str, fold_name: str, X_train, y_train, X_test, y_test, C_cat, C_num, estimator, normalize, select, oversample, random_state):
    try:
        start_time = time.time()
        if normalize:
            X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
            X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values
            
            scaler = StandardScaler().fit(X_train_N)
            X_train_N = scaler.transform(X_train_N)
            X_test_N = scaler.transform(X_test_N)
        
            X_train = pd.DataFrame(
                np.concatenate((X_train_C, X_train_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )
            X_test = pd.DataFrame(
                np.concatenate((X_test_C, X_test_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )
            
        if select:
            # # Removing low variance features
            # X_train = exclude_low_variance(X_train)
            # X_test = X_test[X_train.columns]  # Keep only the selected features in the test set

            # #Removing highly correlated features
            # X_train = remove_pairwise_corr(X_train, outcome_variable= y_train)
            # X_test = X_test[X_train.columns]  # Keep only the selected features in the test set

            if isinstance(select, SelectFromModel):
                select = [select]
                
            for i, s in enumerate(select):
                C = np.asarray(X_train.columns)
                M = s.fit(X=X_train.values, y=y_train).get_support()
                C_sel = C[M]
                C_cat = C_cat[np.isin(C_cat, C_sel)]
                C_num = C_num[np.isin(C_num, C_sel)]
                
                X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
                X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values


                X_train = pd.DataFrame(
                    np.concatenate((X_train_C, X_train_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )
                X_test = pd.DataFrame(
                    np.concatenate((X_test_C, X_test_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )

        if oversample:
            if len(C_cat) > 0:
                sampler = SMOTENC(categorical_features=[X_train.columns.get_loc(c) for c in C_cat], random_state=random_state)
            else:
                sampler = SMOTE(random_state=random_state)
            X_train, y_train = sampler.fit_resample(X_train, y_train)

        estimator = clone(estimator).fit(X_train, y_train)
        y_pred = estimator.predict_proba(X_test)[:, 1]
        auc_score = roc_auc_score(y_test, y_pred, average=None)

        result = FoldResult(
            name=fold_name,
            metrics={'AUC': auc_score},
            duration=time.time() - start_time
        )
        log(f'Training completed for {fold_name} with AUC: {auc_score}')
        return result

    except Exception as e:
        log(f'Error in {fold_name}: {traceback.format_exc()}')
        return None

def perform_cross_validation(X, y, groups, estimator, normalize=False, select=None, oversample=False, random_state=None):
    if not ray.is_initialized():
        ray.init()

    futures = []
    splitter = LeaveOneGroupOut()  # Or any other CV strategy
    for idx, (train_idx, test_idx) in enumerate(splitter.split(X, y, groups)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        C_cat = np.asarray(sorted(cats))
        C_num = np.asarray(sorted(X.columns[~X.columns.isin(C_cat)]))

        job = train_fold.remote('path_to_results', f'Fold_{idx}', X_train, y_train, X_test, y_test, C_cat, C_num, estimator, normalize, select, oversample, random_state)
        futures.append(job)

    results = ray.get(futures)
    return results

with on_ray():
    SELECT_LASSO = SelectFromModel(
            estimator=LogisticRegression(
            penalty='l1' 
            ,solver='liblinear'
            , C=1, random_state=RANDOM_STATE, max_iter=4000
        ),
        threshold = 0.005
    )
    # Example usage
    estimator = EvXGBClassifier(
        random_state=RANDOM_STATE, 
        eval_metric='logloss', 
        eval_size=0.2,
        early_stopping_rounds=10, 
        objective='binary:logistic', 
        verbosity=0,
        learning_rate=0.01,
    )  
    results = perform_cross_validation(X, y, groups, estimator, normalize=True, select=[SELECT_LASSO], oversample=True, random_state=42)
    auc_values = [results[i].metrics['AUC'] for i in range(len(results))]
    mean_auc = np.mean(auc_values)
    print(mean_auc)

2024-04-23 17:36:55,503	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(train_fold pid=1110891) Training completed for Fold_15 with AUC: 0.5727272727272728
(train_fold pid=1110892) Training completed for Fold_7 with AUC: 0.5609756097560976 [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(train_fold pid=1110900) Training completed for Fold_5 with AUC: 0.5515873015873015 [repeated 4x across cluster]
(train_fold pid=1110896) Training completed for Fold_13 with AUC: 0.3111111111111111 [repeated 5x across cluster]
(train_fold pid=1110899) Training completed for Fold_8 with AUC: 0.6799999999999999 [repeated 3x across cluster]
(train_fold pid=1110891) Training completed for Fold_16 with AUC: 0.43589743589743596
(train_fold pid=1110893) Training completed for Fold_18 with AUC: 0.6749999999999999
(train_fold pid=1110905) Training completed for Fold_21 with AUC: 0.5286195286195287 [repeated 2

In [6]:
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, hp, fmin, tpe
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from imblearn.over_sampling import SMOTE
from ray import tune
from ray.tune import with_parameters
from ray.tune.schedulers import HyperBandScheduler
from ray.tune.search.hyperopt import HyperOptSearch
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
import time
import traceback
import ray
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.metrics import roc_auc_score
from dataclasses import dataclass

@dataclass
class FoldResult:
    name: str
    metrics: dict
    duration: float

def log(message: str):
    print(message)  # Simple logging to stdout or enhance as needed

def train_fold(dir_result: str, fold_name: str, X_train, y_train, X_test, y_test, C_cat, C_num, estimator, normalize, select, oversample, random_state):
    try:
        start_time = time.time()
        if normalize:
            X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
            X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values
            
            scaler = StandardScaler().fit(X_train_N)
            X_train_N = scaler.transform(X_train_N)
            X_test_N = scaler.transform(X_test_N)
        
            X_train = pd.DataFrame(
                np.concatenate((X_train_C, X_train_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )
            X_test = pd.DataFrame(
                np.concatenate((X_test_C, X_test_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )
            
        if select:
            # # Removing low variance features
            # X_train = exclude_low_variance(X_train)
            # X_test = X_test[X_train.columns]  # Keep only the selected features in the test set

            # #Removing highly correlated features
            # X_train = remove_pairwise_corr(X_train, outcome_variable= y_train)
            # X_test = X_test[X_train.columns]  # Keep only the selected features in the test set

            if isinstance(select, SelectFromModel):
                select = [select]
                
            for i, s in enumerate(select):
                C = np.asarray(X_train.columns)
                M = s.fit(X=X_train.values, y=y_train).get_support()
                C_sel = C[M]
                C_cat = C_cat[np.isin(C_cat, C_sel)]
                C_num = C_num[np.isin(C_num, C_sel)]
                
                X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
                X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values


                X_train = pd.DataFrame(
                    np.concatenate((X_train_C, X_train_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )
                X_test = pd.DataFrame(
                    np.concatenate((X_test_C, X_test_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )

        if oversample:
            if len(C_cat) > 0:
                sampler = SMOTENC(categorical_features=[X_train.columns.get_loc(c) for c in C_cat], random_state=random_state)
            else:
                sampler = SMOTE(random_state=random_state)
            X_train, y_train = sampler.fit_resample(X_train, y_train)

        estimator = clone(estimator).fit(X_train, y_train)
        y_pred = estimator.predict_proba(X_test)[:, 1]
        auc_score = roc_auc_score(y_test, y_pred, average=None)

        result = FoldResult(
            name=fold_name,
            metrics={'AUC': auc_score},
            duration=time.time() - start_time
        )
        log(f'Training completed for {fold_name} with AUC: {auc_score}')
        return result

    except Exception as e:
        log(f'Error in {fold_name}: {traceback.format_exc()}')
        return None

def perform_cross_validation(X, y, groups, estimator, normalize=False, select=None, oversample=False, random_state=None):
    if not ray.is_initialized():
        ray.init()

    futures = []
    splitter = LeaveOneGroupOut()  # Or any other CV strategy
    for idx, (train_idx, test_idx) in enumerate(splitter.split(X, y, groups)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        C_cat = np.asarray(sorted(cats))
        C_num = np.asarray(sorted(X.columns[~X.columns.isin(C_cat)]))

        job = train_fold('path_to_results', f'Fold_{idx}', X_train, y_train, X_test, y_test, C_cat, C_num, estimator, normalize, select, oversample, random_state)
        futures.append(job)

    results = futures
    return results

def objective(params, X, y, groups):
    SELECT_LASSO = SelectFromModel(
            estimator=LogisticRegression(
            penalty='l1' 
            ,solver='liblinear'
            , C=1, random_state=RANDOM_STATE, max_iter=4000
        ),
        threshold = 0.005
    )
    # Example usage
    estimator = EvXGBClassifier(
        random_state=RANDOM_STATE, 
        eval_metric='logloss', 
        eval_size=0.2,
        objective='binary:logistic', 
        verbosity=0,
        **params
    )  

    results = perform_cross_validation(X, y, groups, estimator, normalize=True, select=[SELECT_LASSO], oversample=True, random_state=42)
    auc_values = [results[i].metrics['AUC'] for i in range(len(results))]

    mean_auc = np.mean(auc_values)
    return {'loss': -mean_auc, 'auc': mean_auc, 'status': STATUS_OK}

space = {
    'max_depth': hp.choice('max_depth', range(3, 11)),
    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'n_estimators': hp.choice('n_estimators', [100, 250, 500]),
    'reg_lambda': hp.uniform('reg_lambda', 0.5, 5.0),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 2.0),
    'num_parallel_tree': hp.choice('num_parallel_tree', [1, 10, 20]),
    'early_stopping_rounds': hp.choice('early_stopping_rounds', [10, 30, 50]),
}

# Setup HyperOpt search with Ray Tune
algo = HyperOptSearch(space, metric="auc", mode="max")

# Define the scheduler for early stopping
scheduler = HyperBandScheduler(time_attr="training_iteration", metric="auc", mode="max")
with on_ray():
    # Assuming X, y, and groups are predefined datasets
    analysis = tune.run(
        with_parameters(objective, X=X, y=y, groups=groups),
        num_samples=100,
        search_alg=algo,
        resources_per_trial={"cpu": 1},
        verbose=1,
        scheduler=scheduler
        
    )

(objective pid=1112258) Training completed for Fold_0 with AUC: 0.5329153605015674
(objective pid=1112403) Training completed for Fold_0 with AUC: 0.5329153605015674 [repeated 3x across cluster]
(objective pid=1112432) Training completed for Fold_0 with AUC: 0.5329153605015674
(objective pid=1112574) Training completed for Fold_0 with AUC: 0.5329153605015674
(objective pid=1112545) Training completed for Fold_0 with AUC: 0.5329153605015674
(objective pid=1112320) Training completed for Fold_1 with AUC: 0.4153439153439154
(objective pid=1112258) Training completed for Fold_1 with AUC: 0.4153439153439154
(objective pid=1112403) Training completed for Fold_1 with AUC: 0.4153439153439154
(objective pid=1112289) Training completed for Fold_1 with AUC: 0.4153439153439154
(objective pid=1112574) Training completed for Fold_1 with AUC: 0.4153439153439154
(objective pid=1112432) Training completed for Fold_1 with AUC: 0.4153439153439154
(objective pid=1112545) Training completed for Fold_1 with

2024-04-23 19:11:39,035	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


(objective pid=1112516) Training completed for Fold_43 with AUC: 0.4979166666666667 [repeated 2x across cluster]
(objective pid=1112963) Training completed for Fold_6 with AUC: 0.4734042553191489


2024-04-23 19:11:49,083	INFO tune.py:1148 -- Total run time: 5379.57 seconds (5369.43 seconds for the tuning loop).
2024-04-23 19:11:49,084	WARNING tune.py:1163 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2024-04-23 19:11:49,160	WARNING experiment_analysis.py:916 -- Failed to read the results for 11 trials:
- /home/panyu/ray_results/objective_2024-04-23_17-42-09/objective_9042ea90_8_colsample_bylevel=0.7380,colsample_bytree=0.5183,early_stopping_rounds=50,gamma=0.3072,learning_rate=0.0773,m_2024-04-23_17-42-20
- /home/panyu/ray_results/objective_2024-04-23_17-42-09/objective_e0abbe7c_11_colsample_bylevel=0.9430,colsample_bytree=0.9743,early_stopping_rounds=30,gamma=0.4965,learning_rate=0.0857,_2024-04-23_17-42-27
- /home/panyu/ray_results/objective_2024-04-23_17-42-09/objective_c5af3588_14_colsample_bylevel=0.6406,colsample_bytree=0.8620,early_stopping_rounds=30,gamma=0.1081,learning_rate=0.0310,_2024-04-23

In [7]:
# Explicitly specify the metric and mode when fetching the best trial
best_trial = analysis.get_best_trial(metric="loss", mode="min")
print("Best trial config: {}".format(best_trial.config))
print("Best trial AUC: {}".format(best_trial.last_result["loss"]))

Best trial config: {'colsample_bylevel': 0.591680745868391, 'colsample_bytree': 0.6743168724873578, 'early_stopping_rounds': 30, 'gamma': 0.16718407941313662, 'learning_rate': 0.03451088913601569, 'max_depth': 9, 'min_child_weight': 4.0, 'n_estimators': 500, 'num_parallel_tree': 10, 'reg_alpha': 0.9674653063681278, 'reg_lambda': 3.3752464686948005, 'subsample': 0.747979943335319}
Best trial AUC: -0.5189142967578859


In [10]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, hp, fmin, tpe
from sklearn.model_selection import train_test_split, LeaveOneGroupOut
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from imblearn.over_sampling import SMOTENC
from ray import tune
from ray.tune import with_parameters
from ray.tune.schedulers import HyperBandScheduler
from ray.tune.search.hyperopt import HyperOptSearch
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.base import clone



SELECT_SVC = SelectFromModel(
#     estimator=LinearSVC(
#         penalty='l1',
#         loss='squared_hinge',
#         dual=False,
#         tol=1e-3,
#         C=1e-2,
#         max_iter=5000,
#         random_state=RANDOM_STATE
#     ),
#     threshold=1e-5
    
        estimator=LogisticRegression(
        penalty='l1' 
        ,solver='liblinear'
        , C=1, random_state=42, max_iter=4000
    ),
    threshold = 0.005
)


def objective(params, X, y, groups):
    scaler = StandardScaler()
    logo = LeaveOneGroupOut()
    auc_scores = []

    for train_idx, test_idx in logo.split(X, y, groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        ##########################
        normalize = True
        select = [SELECT_SVC]
        oversample = True
        random_state = 42


        C_cat = np.asarray(sorted(cats))
        C_num = np.asarray(sorted(X.columns[~X.columns.isin(C_cat)]))


        if normalize:
            X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
            X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values
            
            scaler = StandardScaler().fit(X_train_N)
            X_train_N = scaler.transform(X_train_N)
            X_test_N = scaler.transform(X_test_N)
        
            X_train = pd.DataFrame(
                np.concatenate((X_train_C, X_train_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )
            X_test = pd.DataFrame(
                np.concatenate((X_test_C, X_test_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )
            
        if select:
            # # Removing low variance features
            # X_train = exclude_low_variance(X_train)
            # X_test = X_test[X_train.columns]  # Keep only the selected features in the test set

            # #Removing highly correlated features
            # X_train = remove_pairwise_corr(X_train, outcome_variable= y_train)
            # X_test = X_test[X_train.columns]  # Keep only the selected features in the test set

            if isinstance(select, SelectFromModel):
                select = [select]
                
            for i, s in enumerate(select):
                C = np.asarray(X_train.columns)
                M = s.fit(X=X_train.values, y=y_train).get_support()
                C_sel = C[M]
                C_cat = C_cat[np.isin(C_cat, C_sel)]
                C_num = C_num[np.isin(C_num, C_sel)]
                
                X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
                X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values


                X_train = pd.DataFrame(
                    np.concatenate((X_train_C, X_train_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )
                X_test = pd.DataFrame(
                    np.concatenate((X_test_C, X_test_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )

        if oversample:
            if len(C_cat) > 0:
                sampler = SMOTENC(categorical_features=[X_train.columns.get_loc(c) for c in C_cat], random_state=random_state)
            else:
                sampler = SMOTE(random_state=random_state)
            X_train, y_train = sampler.fit_resample(X_train, y_train)

        ##########################


        # # Split training data for early stopping
        # X_train_sub, X_val, y_train_sub, y_val = train_test_split(
        #     X_train, y_train, test_size=0.2, random_state=int(params['random_state'])
        # )
        
        # # Classifier
        # clf = xgb.XGBClassifier(
        #     objective='binary:logistic',
        #     eval_metric='auc',
        #     verbosity=0,
        #     tree_method='hist',
        #     **params
        # )
        # clf.fit(
        #     X_train_sub, y_train_sub,
        #     eval_set=[(X_val, y_val)],
        #     verbose=False
        # )

        # best_iteration = clf.best_iteration
        # y_pred = clf.predict_proba(X_test, iteration_range=(0, best_iteration + 1))[:, 1]
        # auc_score = roc_auc_score(y_test, y_pred)
        estimator = EvXGBClassifier(
        eval_metric='logloss', 
        eval_size=0.2,
        method='hist-gpu',
        objective='binary:logistic', 
        verbosity=0,
        **params
        )  
        estimator = clone(estimator).fit(X_train, y_train)
        y_pred = estimator.predict_proba(X_test)[:, 1]
        auc_score = roc_auc_score(y_test, y_pred, average=None)
        
        auc_scores.append(auc_score)

    mean_auc = np.mean(auc_scores)
    return {'loss': -mean_auc, 'auc': mean_auc, 'status': STATUS_OK}

space = {
    'max_depth': hp.choice('max_depth', range(3, 11)),
    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'n_estimators': hp.choice('n_estimators', [100, 250, 500]),
    'reg_lambda': hp.uniform('reg_lambda', 0.5, 5.0),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 2.0),
    'num_parallel_tree': hp.choice('num_parallel_tree', [1, 10, 20]),
    'early_stopping_rounds': hp.choice('early_stopping_rounds', [10, 30, 50]),
    'random_state': hp.choice('random_state', [42])
}

# Setup HyperOpt search with Ray Tune
algo = HyperOptSearch(space, metric="auc", mode="max")

# Define the scheduler for early stopping
scheduler = HyperBandScheduler(time_attr="training_iteration", metric="auc", mode="max")
with on_ray():
    # Assuming X, y, and groups are predefined datasets
    analysis = tune.run(
        with_parameters(objective, X=X, y=y, groups=groups),
        num_samples=100,
        search_alg=algo,
        # resources_per_trial={"cpu": 1},
        verbose=1,
        scheduler=scheduler
        
    )

2024-04-24 13:33:12,519	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-04-24 13:33:22,555	INFO tune.py:1148 -- Total run time: 1359.57 seconds (1349.52 seconds for the tuning loop).
2024-04-24 13:33:22,556	WARNING tune.py:1163 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2024-04-24 13:33:22,560	WARNING experiment_analysis.py:916 -- Failed to read the results for 29 trials:
- /home/iclab/ray_results/objective_2024-04-24_13-10-42/objective_44c3a743_1_colsample_bylevel=0.6350,colsample_bytree=0.9829,early_stopping_rounds=30,gamma=0.3998,learning_rate=0.0322,m_2024-04-24_13-10-43
- /home/iclab/ray_results/objective_2024-04-24_13-10-42/objective_afb11107_2_colsample_bylevel=0.7698,colsample_bytree=0.7306,early_stopping_rounds=10,gamma=0.3962,learn

In [7]:
# Explicitly specify the metric and mode when fetching the best trial
best_trial = analysis.get_best_trial(metric="loss", mode="min")
print("Best trial config: {}".format(best_trial.config))
print("Best trial AUC: {}".format(best_trial.last_result["loss"]))

Best trial config: {'colsample_bylevel': 0.681073126404533, 'colsample_bytree': 0.6443005293308781, 'early_stopping_rounds': 30, 'gamma': 0.28891067270936654, 'learning_rate': 0.08848040162909752, 'max_depth': 8, 'min_child_weight': 2.0, 'n_estimators': 100, 'num_parallel_tree': 1, 'random_state': 42, 'reg_alpha': 1.0787212485467532, 'reg_lambda': 1.856673415371724, 'subsample': 0.67266853756236}
Best trial AUC: -0.5189142967578859
